# Домашнее задание №2: Синтез речи


In [ ]:
# Установка зависимостей для сборки
# setuptools нужен для distutils (Python 3.12+)
%pip install setuptools wheel build Cython

# Импорты для работы с путями и процессами
import os
import subprocess
import sys

# Установка piper из локальной папки
# ВАЖНО: После установки может потребоваться перезапуск ядра Jupyter
LOCAL_PIPER_PATH = "/tf/piper1-gpl"

print("Установка piper из локальной папки...")
print("Это может занять несколько минут из-за компиляции C-расширений...")

# Установка piper
%pip install -e /tf/piper1-gpl

# Проверяем, установился ли espeakbridge
import subprocess
import sys

try:
    from piper import espeakbridge
    print("✓ espeakbridge успешно установлен")
except ImportError as e:
    print(f"✗ espeakbridge не найден: {e}")
    print("\nПопытка автоматической сборки espeakbridge...")
    
    # Пробуем собрать espeakbridge
    try:
        result = subprocess.run(
            [sys.executable, "setup.py", "build_ext", "--inplace"],
            cwd="/tf/piper1-gpl",
            capture_output=True,
            text=True,
            timeout=300
        )
        if result.returncode == 0:
            print("✓ Сборка завершена, переустанавливаем...")
            subprocess.run([sys.executable, "-m", "pip", "install", "-e", "/tf/piper1-gpl"], check=False)
            
            # Проверяем снова
            try:
                from piper import espeakbridge
                print("✓ espeakbridge успешно собран и установлен")
            except ImportError:
                print("✗ espeakbridge все еще не найден после сборки")
                print("\n⚠️  Выполните в терминале:")
                print("  cd /tf/piper1-gpl")
                print("  python setup.py build_ext --inplace")
                print("  pip install -e .")
        else:
            print(f"✗ Ошибка сборки: {result.stderr}")
            print("\n⚠️  Возможно, нужны системные зависимости:")
            print("  sudo apt-get install espeak-ng libespeak-ng-dev")
    except Exception as build_error:
        print(f"✗ Не удалось собрать автоматически: {build_error}")
        print("\n⚠️  Выполните в терминале:")
        print("  cd /tf/piper1-gpl")
        print("  python setup.py build_ext --inplace")
        print("  pip install -e .")

# Проверяем и собираем monotonic_align (критически важно для обучения)
print("\n" + "="*60)
print("ПРОВЕРКА И СБОРКА MONOTONIC_ALIGN")
print("="*60)

try:
    from piper.train.vits.monotonic_align import monotonic_align
    from piper.train.vits.monotonic_align.monotonic_align.core import maximum_path_c
    print("✓ monotonic_align успешно установлен")
except (ImportError, ModuleNotFoundError) as e:
    print(f"✗ monotonic_align не найден: {e}")
    print("\nПопытка автоматической сборки monotonic_align...")
    
    # Проверяем наличие скрипта сборки
    build_script = os.path.join(LOCAL_PIPER_PATH, "build_monotonic_align.sh")
    monotonic_dir = os.path.join(LOCAL_PIPER_PATH, "src", "piper", "train", "vits", "monotonic_align")
    
    if os.path.exists(build_script):
        print(f"  Найден скрипт сборки: {build_script}")
        try:
            result = subprocess.run(
                ["bash", build_script],
                cwd=LOCAL_PIPER_PATH,
                capture_output=True,
                text=True,
                timeout=600
            )
            if result.returncode == 0:
                print("✓ Сборка monotonic_align завершена")
                # Переустанавливаем piper
                subprocess.run([sys.executable, "-m", "pip", "install", "-e", LOCAL_PIPER_PATH], check=False)
            else:
                print(f"✗ Ошибка сборки скриптом: {result.stderr[:500] if result.stderr else 'Неизвестная ошибка'}")
                print(f"   Вывод: {result.stdout[:500] if result.stdout else 'Нет вывода'}")
        except Exception as build_error:
            print(f"✗ Ошибка при запуске скрипта: {build_error}")
    
    # Пробуем собрать через setup.py
    if os.path.exists(monotonic_dir):
        print(f"\n  Пробуем собрать через setup.py...")
        try:
            result = subprocess.run(
                [sys.executable, "setup.py", "build_ext", "--inplace"],
                cwd=LOCAL_PIPER_PATH,
                capture_output=True,
                text=True,
                timeout=600
            )
            if result.returncode == 0:
                print("✓ Сборка через setup.py завершена, переустанавливаем...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-e", LOCAL_PIPER_PATH], check=False)
            else:
                print(f"✗ Ошибка сборки через setup.py: {result.stderr[:500] if result.stderr else 'Неизвестная ошибка'}")
        except Exception as build_error:
            print(f"✗ Ошибка сборки: {build_error}")
    
    # Проверяем снова
    try:
        from piper.train.vits.monotonic_align import monotonic_align
        from piper.train.vits.monotonic_align.monotonic_align.core import maximum_path_c
        print("\n✓ monotonic_align успешно собран и установлен")
    except (ImportError, ModuleNotFoundError):
        print("\n✗ monotonic_align все еще не найден после сборки")
        print("\n⚠️  Выполните в терминале:")
        print("  cd /tf/piper1-gpl")
        print("  pip install setuptools Cython")
        print("  bash build_monotonic_align.sh")
        print("  # или")
        print("  python setup.py build_ext --inplace")
        print("  pip install -e .")
        print("\n⚠️  Возможно, нужны системные зависимости:")
        print("  sudo apt-get install build-essential python3-dev")

# Установка nvidia-ml-py для замены устаревшего pynvml
%pip install nvidia-ml-py

# Проверка установки
try:
    import pynvml
    print("⚠️  pynvml все еще установлен - он используется torch")
    print("   Предупреждение о pynvml исходит из torch, не из нашего кода")
    print("   nvidia-ml-py установлен для будущего использования")
except ImportError:
    print("✓ pynvml не установлен")

try:
    import nvidia_ml_py3 as nvml
    print("✓ nvidia-ml-py успешно установлен")
except ImportError:
    print("⚠️  nvidia-ml-py не установлен, но это не критично")

%pip install tensorboard
%pip install onnx
%pip install onnxruntime

# Проверка версии torch
import torch
print(f"\nВерсия PyTorch: {torch.__version__}")
print("⚠️  Предупреждение о weight_norm исходит из кода piper,")
print("   его можно исправить только в исходниках piper")


# Импорты


In [ ]:
import os
import json
import glob
import random
import re
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import librosa
import soundfile as sf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

SEED = 42

# Устанавливаем seed для всех генераторов случайных чисел
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Для воспроизводимости на GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Устанавливаем seed для Lightning через переменную окружения
import os
os.environ['PL_GLOBAL_SEED'] = str(SEED)

print(f"Seed установлен: {SEED}")
print(f"PL_GLOBAL_SEED: {os.environ.get('PL_GLOBAL_SEED')}")


# Подготовка данных


In [ ]:
DATA_PATH = "/share/audio_data/sova/ytub/raid/nanosemantics/nextcloud/sova_done"

# OUTPUT_DIR делаем абсолютным, чтобы избежать проблем с дублированием путей
OUTPUT_DIR_REL = "./tts_data"
OUTPUT_DIR = os.path.abspath(OUTPUT_DIR_REL)
# Нормализуем путь (убираем лишние компоненты типа ./)
OUTPUT_DIR = os.path.normpath(OUTPUT_DIR)

NATAHA_DIR = "/tf/nspeganov/nataha"

print(f"OUTPUT_DIR установлен: {OUTPUT_DIR}")
print(f"Текущая рабочая директория: {os.getcwd()}")

# Проверяем наличие готовых данных (сначала локальная папка, потом share)
LOCAL_TRAIN_CSV = os.path.join(NATAHA_DIR, "train.csv")
LOCAL_VAL_CSV = os.path.join(NATAHA_DIR, "valid.csv")
LOCAL_TRAIN_DIR = os.path.join(NATAHA_DIR, "train")
LOCAL_VAL_DIR = os.path.join(NATAHA_DIR, "valid")

SHARE_NATAHA_DIR = "/share/tts_data/nataha"
SHARE_TRAIN_CSV = os.path.join(SHARE_NATAHA_DIR, "train.csv")
SHARE_VAL_CSV = os.path.join(SHARE_NATAHA_DIR, "valid.csv")
SHARE_TRAIN_DIR = os.path.join(SHARE_NATAHA_DIR, "train")
SHARE_VAL_DIR = os.path.join(SHARE_NATAHA_DIR, "valid")

# Определяем, какие данные использовать (приоритет локальной папке)
if os.path.exists(LOCAL_TRAIN_CSV) and os.path.exists(LOCAL_VAL_CSV):
    print(f"Найдены готовые данные в локальной папке: {NATAHA_DIR}")
    TRAIN_CSV = LOCAL_TRAIN_CSV
    VAL_CSV = LOCAL_VAL_CSV
    TRAIN_DIR = LOCAL_TRAIN_DIR
    VAL_DIR = LOCAL_VAL_DIR
    USE_PREPARED_DATA = True
elif os.path.exists(SHARE_TRAIN_CSV) and os.path.exists(SHARE_VAL_CSV):
    print(f"Найдены готовые данные в share: {SHARE_NATAHA_DIR}")
    TRAIN_CSV = SHARE_TRAIN_CSV
    VAL_CSV = SHARE_VAL_CSV
    TRAIN_DIR = SHARE_TRAIN_DIR
    VAL_DIR = SHARE_VAL_DIR
    USE_PREPARED_DATA = True
else:
    print("Готовые данные не найдены, будем обрабатывать из SOVA")
    TRAIN_DIR = LOCAL_TRAIN_DIR
    VAL_DIR = LOCAL_VAL_DIR
    USE_PREPARED_DATA = False

TEST_DIR = os.path.join(OUTPUT_DIR, "test")
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

SAMPLE_RATE = 22050
MIN_DURATION = 0.5
MAX_DURATION = 10.0


In [ ]:
def normalize_text(text):
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def trim_silence(audio, sr, threshold_db=-40):
    try:
        trimmed, _ = librosa.effects.trim(audio, top_db=abs(threshold_db))
        return trimmed
    except:
        return audio

def load_audio_info(audio_path):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        duration = len(y) / sr
        return duration, sr, y
    except Exception as e:
        print(f"Ошибка при загрузке {audio_path}: {e}")
        return None, None, None

def prepare_tts_dataset(base_path, parts=['part_0', 'part_1', 'part_2'], max_files=None, trim_silence_audio=True):
    import time
    dataset = []
    audio_extensions = ['.wav', '.mp3', '.flac', '.ogg']
    audio_files = []
    
    print("Поиск аудиофайлов...")
    search_start = time.time()
    
    for part in parts:
        part_path = os.path.join(base_path, part)
        if not os.path.exists(part_path):
            print(f"Предупреждение: часть {part} не найдена")
            continue
        
        print(f"\nОбработка {part}...")
        part_start = time.time()
        
        # Сначала собираем все поддиректории для оценки прогресса
        all_dirs = []
        print("  Сканирование структуры директорий...")
        for root, dirs, files in os.walk(part_path):
            all_dirs.append(root)
        
        print(f"  Найдено {len(all_dirs)} поддиректорий")
        
        # Поиск файлов с прогресс-баром
        part_files = []
        ext_counts = {ext: 0 for ext in audio_extensions}
        
        pbar_search = tqdm(all_dirs, desc=f"Поиск в {part}", unit="дир", ncols=100, leave=False)
        for root in pbar_search:
            try:
                for file in os.listdir(root):
                    file_path = os.path.join(root, file)
                    if os.path.isfile(file_path):
                        file_ext = os.path.splitext(file)[1].lower()
                        if file_ext in audio_extensions:
                            part_files.append(file_path)
                            ext_counts[file_ext] += 1
                            pbar_search.set_postfix({
                                'найдено': len(part_files),
                                'wav': ext_counts['.wav'],
                                'mp3': ext_counts['.mp3']
                            })
            except (PermissionError, OSError) as e:
                continue
        
        audio_files.extend(part_files)
        part_time = time.time() - part_start
        print(f"  {part}: найдено {len(part_files)} файлов за {part_time:.1f}с ({part_time/60:.1f} мин)")
        for ext, count in ext_counts.items():
            if count > 0:
                print(f"    {ext}: {count} файлов")
    
    search_time = time.time() - search_start
    
    if max_files and len(audio_files) > max_files:
        print(f"\nОграничение: было {len(audio_files)} файлов, оставляем {max_files}")
        audio_files = audio_files[:max_files]
    
    print(f"\n{'='*60}")
    print(f"Поиск завершен: найдено {len(audio_files)} аудиофайлов")
    print(f"Время поиска: {search_time:.1f}с ({search_time/60:.1f} мин)")
    print(f"{'='*60}\n")
    
    print("Начинаю обработку файлов...")
    
    start_time = time.time()
    processed = 0
    skipped_no_text = 0
    skipped_duration = 0
    skipped_trim = 0
    
    pbar = tqdm(audio_files, desc="Подготовка данных", unit="файл", ncols=100)
    for audio_path in pbar:
        duration, sr, y = load_audio_info(audio_path)
        if duration is None:
            continue
        
        if duration < MIN_DURATION or duration > MAX_DURATION:
            skipped_duration += 1
            continue
        
        text_path = audio_path.rsplit('.', 1)[0] + '.txt'
        if not os.path.exists(text_path):
            skipped_no_text += 1
            continue
        
        with open(text_path, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        
        text = normalize_text(text)
        
        if len(text) > 0:
            resampled_audio = librosa.resample(y, orig_sr=sr, target_sr=SAMPLE_RATE) if sr != SAMPLE_RATE else y
            
            if trim_silence_audio:
                resampled_audio = trim_silence(resampled_audio, SAMPLE_RATE)
                if len(resampled_audio) < SAMPLE_RATE * MIN_DURATION:
                    skipped_trim += 1
                    continue
            
            dataset.append({
                "audio": resampled_audio,
                "text": text,
                "duration": len(resampled_audio) / SAMPLE_RATE,
                "path": audio_path
            })
            processed += 1
        
        elapsed = time.time() - start_time
        pbar.set_postfix({
            'Обработано': processed,
            'Пропущено': skipped_no_text + skipped_duration + skipped_trim,
            'Время': f"{elapsed:.1f}с"
        })
    
    elapsed_total = time.time() - start_time
    print(f"\nПодготовка завершена!")
    print(f"  Обработано записей: {len(dataset)}")
    print(f"  Пропущено (нет текста): {skipped_no_text}")
    print(f"  Пропущено (длительность): {skipped_duration}")
    print(f"  Пропущено (после обрезки): {skipped_trim}")
    print(f"  Время обработки: {elapsed_total:.1f} секунд ({elapsed_total/60:.1f} минут)")
    print(f"  Скорость: {len(audio_files)/elapsed_total:.1f} файлов/сек")
    
    return dataset


In [ ]:
print("=" * 60)
print("ПОДГОТОВКА ДАННЫХ")
print("=" * 60)

if USE_PREPARED_DATA:
    print(f"\nИспользуем готовые данные из CSV файлов")
    print(f"  Train CSV: {TRAIN_CSV}")
    print(f"  Val CSV: {VAL_CSV}")
    print(f"  Train Dir: {TRAIN_DIR}")
    print(f"  Val Dir: {VAL_DIR}")
    
    # Загружаем информацию из CSV (только для статистики, сами аудио не загружаем)
    train_data = []
    val_data = []
    
    print("\nЧтение train.csv...")
    with open(TRAIN_CSV, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Загрузка train", unit="строка"):
            line = line.strip()
            if '|' in line:
                audio_file, text = line.split('|', 1)
                train_data.append({
                    'audio_file': audio_file,
                    'text': text,
                    'path': os.path.join(TRAIN_DIR, audio_file)
                })
    
    print(f"Загружено {len(train_data)} записей из train.csv")
    
    print("\nЧтение valid.csv...")
    with open(VAL_CSV, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Загрузка valid", unit="строка"):
            line = line.strip()
            if '|' in line:
                audio_file, text = line.split('|', 1)
                val_data.append({
                    'audio_file': audio_file,
                    'text': text,
                    'path': os.path.join(VAL_DIR, audio_file)
                })
    
    print(f"Загружено {len(val_data)} записей из valid.csv")
    
    # Для теста используем часть валидационных данных
    test_size = int(0.1 * len(val_data))
    test_data = val_data[:test_size]
    val_data = val_data[test_size:]
    
    print(f"\nРазделение данных:")
    print(f"  Train: {len(train_data)} записей")
    print(f"  Val: {len(val_data)} записей")
    print(f"  Test: {len(test_data)} записей")
    
    print("\n" + "=" * 60)
    print("ДАННЫЕ ЗАГРУЖЕНЫ ИЗ ГОТОВЫХ CSV")
    print("=" * 60)
    
else:
    print("\nОбрабатываем данные из SOVA...")
    dataset = prepare_tts_dataset(DATA_PATH, parts=['part_0', 'part_1', 'part_2'])
    
    print("\n" + "=" * 60)
    print("РАЗДЕЛЕНИЕ ДАННЫХ")
    print("=" * 60)
    
    print("Перемешивание данных...")
    random.shuffle(dataset)
    
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    
    train_data = dataset[:train_size]
    val_data = dataset[train_size:train_size + val_size]
    test_data = dataset[train_size + val_size:]
    
    print(f"\nРазделение данных:")
    print(f"  Train: {len(train_data)} записей ({100*len(train_data)/len(dataset):.1f}%)")
    print(f"  Val: {len(val_data)} записей ({100*len(val_data)/len(dataset):.1f}%)")
    print(f"  Test: {len(test_data)} записей ({100*len(test_data)/len(dataset):.1f}%)")
    
    print("\nВычисление статистики по длительности...")
    durations = [item['duration'] for item in tqdm(dataset, desc="Обработка длительностей", unit="запись", leave=False)]
    
    print(f"\nСтатистика по длительности:")
    print(f"  Минимум: {min(durations):.2f} сек")
    print(f"  Максимум: {max(durations):.2f} сек")
    print(f"  Среднее: {np.mean(durations):.2f} сек")
    print(f"  Медиана: {np.median(durations):.2f} сек")
    print(f"  Общая длительность: {sum(durations)/3600:.2f} часов")
    
    print("\n" + "=" * 60)
    print("ПОДГОТОВКА ДАННЫХ ЗАВЕРШЕНА")
    print("=" * 60)


# Сохранение подготовленных данных


In [ ]:
def save_dataset_csv(data, output_dir, csv_path):
    os.makedirs(output_dir, exist_ok=True)
    
    with open(csv_path, 'w', encoding='utf-8') as f:
        for i, item in enumerate(tqdm(data, desc="Сохранение", unit="файл")):
            audio_filename = f"{i:06d}.wav"
            audio_path = os.path.join(output_dir, audio_filename)
            
            sf.write(audio_path, item['audio'], SAMPLE_RATE)
            
            text = item['text'].replace('|', ' ').replace('\n', ' ').strip()
            f.write(f"{audio_filename}|{text}\n")
    
    print(f"CSV сохранен: {csv_path}")
    return csv_path

if USE_PREPARED_DATA:
    print("Используем готовые CSV файлы, сохранение не требуется")
    train_csv = TRAIN_CSV
    val_csv = VAL_CSV
    print(f"  Train CSV: {train_csv}")
    print(f"  Val CSV: {val_csv}")
else:
    print("Сохранение обработанных данных в CSV...")
    train_csv = save_dataset_csv(train_data, TRAIN_DIR, os.path.join(NATAHA_DIR, "train.csv"))
    val_csv = save_dataset_csv(val_data, VAL_DIR, os.path.join(NATAHA_DIR, "valid.csv"))
    
    print(f"\nCSV файлы сохранены:")
    print(f"  Train: {train_csv}")
    print(f"  Val: {val_csv}")


# Создание конфигурационного файла


In [ ]:
config_path = os.path.join(OUTPUT_DIR, "nata_config.json")

config = {
    "audio": {
        "sample_rate": SAMPLE_RATE
    },
    "espeak": {
        "voice": "ru"
    },
    "phoneme_type": "espeak",
    "num_symbols": 256,
    "num_speakers": 1,
    "inference": {
        "noise_scale": 0.667,
        "length_scale": 1.0,
        "noise_w": 0.8
    },
    "hop_length": 256,
    "piper_version": "1.3.0"
}

with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"Конфигурация сохранена: {config_path}")


In [ ]:
# Загрузка базового чекпоинта


# Загрузка базового чекпоинта


In [ ]:
CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

base_checkpoint_url = "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/ru/ru_RU/ruslan/medium/epoch=2436-step=1724372.ckpt"
base_checkpoint_path = os.path.join(CHECKPOINT_DIR, "epoch=2436-step=1724372.ckpt")

if not os.path.exists(base_checkpoint_path):
    print("Загрузка базового чекпоинта...")
    import urllib.request
    urllib.request.urlretrieve(base_checkpoint_url, base_checkpoint_path)
    print(f"Чекпоинт загружен: {base_checkpoint_path}")
else:
    print(f"Чекпоинт уже существует: {base_checkpoint_path}")


In [ ]:
import subprocess

cache_dir = os.path.join(OUTPUT_DIR, "nata_cache")
os.makedirs(cache_dir, exist_ok=True)

log_dir = os.path.join(OUTPUT_DIR, "lightning_logs")
os.makedirs(log_dir, exist_ok=True)

print("Запуск обучения...")
print("Команда обучения будет выполнена в следующей ячейке")

train_cmd = [
    "python3", "-m", "piper.train", "fit",
    "--data.voice_name", "nata",
    "--data.csv_path", train_csv,
    "--data.audio_dir", TRAIN_DIR,
    "--model.sample_rate", str(SAMPLE_RATE),
    "--data.espeak_voice", "ru",
    "--data.cache_dir", cache_dir,
    "--data.config_path", config_path,
    "--data.batch_size", "16",
    "--ckpt_path", base_checkpoint_path
]

print(" ".join(train_cmd))


In [ ]:
import subprocess
import sys

# Проверка установки piper
print("Проверка установки piper...")
try:
    import piper
    print(f"✓ Piper установлен: {piper.__file__}")
except ImportError:
    print("✗ Piper не установлен, пробуем добавить локальный путь...")
    LOCAL_PIPER_PATH = "/tf/piper1-gpl"
    if os.path.exists(LOCAL_PIPER_PATH):
        sys.path.insert(0, LOCAL_PIPER_PATH)
        print(f"  Добавлен путь: {LOCAL_PIPER_PATH}")
        try:
            import piper
            print(f"✓ Piper загружен из локального пути: {piper.__file__}")
        except ImportError:
            print("✗ Не удалось загрузить piper, проверьте установку")
    else:
        print("✗ Локальный путь не найден, установите piper: %pip install -e /tf/piper1-gpl")

cache_dir = os.path.join(OUTPUT_DIR, "nata_cache")
os.makedirs(cache_dir, exist_ok=True)

log_dir = os.path.join(OUTPUT_DIR, "lightning_logs")
os.makedirs(log_dir, exist_ok=True)

print("\nНастройка команды обучения...")

# Делаем пути корректными для команды (так как cwd=OUTPUT_DIR)
def make_path_for_cmd(path, base_dir):
    """Делает путь корректным для команды, запускаемой с cwd=base_dir"""
    # Нормализуем base_dir до абсолютного пути
    base_dir_abs = os.path.abspath(os.path.normpath(base_dir))
    
    # Нормализуем path до абсолютного пути
    if os.path.isabs(path):
        path_abs = os.path.abspath(os.path.normpath(path))
    else:
        # Если путь относительный, нормализуем относительно текущей рабочей директории
        # НЕ относительно base_dir, чтобы избежать дублирования
        current_dir = os.getcwd()
        path_abs = os.path.abspath(os.path.normpath(os.path.join(current_dir, path)))
    
    # Проверяем, находится ли путь внутри base_dir
    try:
        # Нормализуем пути для сравнения
        base_dir_norm = os.path.normpath(base_dir_abs)
        path_norm = os.path.normpath(path_abs)
        
        # Проверяем, что path начинается с base_dir
        # Используем os.path.commonpath для более надежной проверки
        try:
            common = os.path.commonpath([base_dir_norm, path_norm])
            if common == base_dir_norm:
                # Путь находится внутри base_dir, делаем его относительным
                rel_path = os.path.relpath(path_norm, base_dir_norm)
                # Проверяем существование
                if os.path.exists(path_abs):
                    return rel_path
        except ValueError:
            # Пути на разных дисках
            pass
        
        # Альтернативная проверка через startswith
        base_dir_with_sep = base_dir_norm + os.sep
        if path_norm == base_dir_norm or path_norm.startswith(base_dir_with_sep):
            rel_path = os.path.relpath(path_norm, base_dir_norm)
            if os.path.exists(path_abs):
                return rel_path
    except (ValueError, OSError) as e:
        # Пути на разных дисках или другая ошибка
        pass
    
    # Иначе используем абсолютный путь
    if os.path.exists(path_abs):
        return path_abs
    
    # Если файл не существует, возвращаем исходный путь (для отладки)
    return path

# Исправляем пути для команды
train_csv_cmd = make_path_for_cmd(train_csv, OUTPUT_DIR)
TRAIN_DIR_cmd = make_path_for_cmd(TRAIN_DIR, OUTPUT_DIR)
cache_dir_cmd = make_path_for_cmd(cache_dir, OUTPUT_DIR)
config_path_cmd = make_path_for_cmd(config_path, OUTPUT_DIR)

# Если локальный путь добавлен, используем его в PYTHONPATH
LOCAL_PIPER_PATH = "/tf/piper1-gpl"
SEED = 42

env = os.environ.copy()
if os.path.exists(LOCAL_PIPER_PATH):
    if 'PYTHONPATH' in env:
        env['PYTHONPATH'] = LOCAL_PIPER_PATH + ':' + env['PYTHONPATH']
    else:
        env['PYTHONPATH'] = LOCAL_PIPER_PATH

# Устанавливаем seed для Lightning через переменную окружения
env['PL_GLOBAL_SEED'] = str(SEED)

# Поиск последнего чекпоинта из предыдущих запусков обучения
print(f"\n{'='*60}")
print("ПОИСК ПОСЛЕДНЕГО ЧЕКПОИНТА")
print(f"{'='*60}")

last_checkpoint = None
last_epoch = 0
MAX_EPOCHS = 2539

# Ищем чекпоинты в lightning_logs
if os.path.exists(log_dir):
    checkpoints = glob.glob(os.path.join(log_dir, "**", "*.ckpt"), recursive=True)
    if checkpoints:
        # Сортируем по времени модификации (последний = самый новый)
        checkpoints.sort(key=os.path.getmtime, reverse=True)
        
        for ckpt in checkpoints:
            # Извлекаем номер эпохи из имени файла (формат: epoch=XXXX-step=YYYY.ckpt)
            match = re.search(r'epoch=(\d+)', os.path.basename(ckpt))
            if match:
                epoch_num = int(match.group(1))
                if epoch_num < MAX_EPOCHS:
                    last_checkpoint = ckpt
                    last_epoch = epoch_num
                    print(f"✓ Найден чекпоинт эпохи {epoch_num}: {ckpt}")
                    break
        
        if last_checkpoint:
            print(f"\nИспользуем последний чекпоинт: эпоха {last_epoch}")
        else:
            print(f"\n⚠️  Последний чекпоинт уже достиг или превысил max_epochs={MAX_EPOCHS}")
            print("   Будет использован базовый чекпоинт")
    else:
        print("Чекпоинты в lightning_logs не найдены")
else:
    print(f"Директория логов не существует: {log_dir}")

# Определяем, какой чекпоинт использовать
if last_checkpoint and last_epoch < MAX_EPOCHS:
    base_checkpoint_path = last_checkpoint
    print(f"\n{'='*60}")
    print("ИСПОЛЬЗУЕМ ПОСЛЕДНИЙ ЧЕКПОИНТ ИЗ ОБУЧЕНИЯ")
    print(f"{'='*60}")
    print(f"Чекпоинт: {base_checkpoint_path}")
    print(f"Эпоха: {last_epoch} (будет продолжено до {MAX_EPOCHS})")
else:
    print(f"\n{'='*60}")
    print("ИСПОЛЬЗУЕМ БАЗОВЫЙ ЧЕКПОИНТ")
    print(f"{'='*60}")

# Проверяем существование чекпоинта и исправляем путь
print(f"\n{'='*60}")
print("ПРОВЕРКА ЧЕКПОИНТА")
print(f"{'='*60}")
print(f"Текущая рабочая директория: {os.getcwd()}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")
print(f"OUTPUT_DIR (abs, norm): {os.path.abspath(os.path.normpath(OUTPUT_DIR))}")
print(f"\nИсходный путь к чекпоинту: {base_checkpoint_path}")
print(f"Исходный путь (abs): {os.path.abspath(base_checkpoint_path) if not os.path.isabs(base_checkpoint_path) else base_checkpoint_path}")

if not os.path.exists(base_checkpoint_path):
    print(f"\n⚠️  ВНИМАНИЕ: Чекпоинт не найден: {base_checkpoint_path}")
    print("   Обучение начнется с нуля")
    checkpoint_path_for_cmd = None
else:
    print(f"\n✓ Чекпоинт найден: {base_checkpoint_path}")
    
    # Для чекпоинта всегда используем абсолютный нормализованный путь
    # Это самый надежный способ избежать проблем с дублированием путей
    checkpoint_path_for_cmd = os.path.abspath(os.path.normpath(base_checkpoint_path))
    
    print(f"\nРезультат обработки пути:")
    print(f"  Абсолютный нормализованный путь: {checkpoint_path_for_cmd}")
    print(f"  Файл существует: {os.path.exists(checkpoint_path_for_cmd)}")
    
    if not os.path.exists(checkpoint_path_for_cmd):
        print(f"\n⚠️  ПРОБЛЕМА: Абсолютный путь не существует!")
        print(f"   Обучение начнется с нуля")
        checkpoint_path_for_cmd = None

train_cmd = [
    "python3", "-m", "piper.train", "fit",
    "--seed_everything", str(SEED),
    "--data.voice_name", "nata",
    "--data.csv_path", train_csv_cmd,
    "--data.audio_dir", TRAIN_DIR_cmd,
    "--model.sample_rate", str(SAMPLE_RATE),
    "--data.espeak_voice", "ru",
    "--data.cache_dir", cache_dir_cmd,
    "--data.config_path", config_path_cmd,
    "--data.batch_size", "16",
    "--data.num_workers", "4",
    "--trainer.max_epochs", "2539",
    "--trainer.enable_progress_bar", "false"
]

# Добавляем чекпоинт только если он существует
if checkpoint_path_for_cmd:
    train_cmd.extend(["--ckpt_path", checkpoint_path_for_cmd])

print("\n" + "="*60)
print("НАСТРОЙКИ ОБУЧЕНИЯ")
print("="*60)
print(f"Рабочая директория (cwd): {OUTPUT_DIR}")
print(f"\nПути к данным:")
print(f"  Train CSV: {train_csv} -> {train_csv_cmd}")
print(f"  Train Dir: {TRAIN_DIR} -> {TRAIN_DIR_cmd}")
print(f"  Cache Dir: {cache_dir} -> {cache_dir_cmd}")
print(f"  Config: {config_path} -> {config_path_cmd}")
if checkpoint_path_for_cmd:
    print(f"  Checkpoint: {base_checkpoint_path} -> {checkpoint_path_for_cmd}")
print(f"\nПеременные окружения:")
print(f"  PYTHONPATH: {env.get('PYTHONPATH', 'не установлен')}")
print(f"  PL_GLOBAL_SEED: {env.get('PL_GLOBAL_SEED', 'не установлен')}")
print(f"\nКоманда обучения:")
print(" ".join(train_cmd))
print("="*60)


In [ ]:
# Используем env с PYTHONPATH для доступа к локальному piper
result = subprocess.run(train_cmd, cwd=OUTPUT_DIR, env=env)
print(f"\nОбучение завершено с кодом: {result.returncode}")
if result.returncode != 0:
    print("\nЕсли возникла ошибка ModuleNotFoundError, попробуйте:")
    print("1. Перезапустить ядро после установки piper")
    print("2. Установить piper: %pip install -e /tf/piper1-gpl")
    print("3. Проверить, что папка /tf/piper1-gpl существует и содержит setup.py")


# Визуализация метрик из TensorBoard


In [ ]:
def plot_training_metrics(log_dir):
    try:
        from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
        
        if not os.path.exists(log_dir):
            print(f"Директория логов не найдена: {log_dir}")
            return
        
        version_dirs = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d)) and d.startswith('version_')]
        if not version_dirs:
            print(f"Логи TensorBoard не найдены в {log_dir}")
            print(f"Содержимое директории: {os.listdir(log_dir) if os.path.exists(log_dir) else 'не существует'}")
            return
        
        latest_version = max(version_dirs, key=lambda x: int(x.split('_')[1]) if x.split('_')[1].isdigit() else 0)
        event_dir = os.path.join(log_dir, latest_version)
        
        print(f"Используем логи из: {event_dir}")
        
        event_acc = EventAccumulator(event_dir)
        event_acc.Reload()
        
        scalar_tags = event_acc.Tags().get('scalars', [])
        print(f"Найдены теги: {scalar_tags}")
        
        if not scalar_tags:
            print("Нет скалярных метрик для визуализации")
            return
        
        # Определяем количество графиков на основе доступных данных
        available_plots = []
        
        # Проверяем наличие loss метрик
        loss_tags = [tag for tag in scalar_tags if 'loss' in tag.lower()]
        has_loss = any('train' in tag.lower() or 'val' in tag.lower() for tag in loss_tags if 'mel' not in tag.lower())
        
        # Проверяем наличие mel_loss метрик
        mel_loss_tags = [tag for tag in scalar_tags if 'mel' in tag.lower() and 'loss' in tag.lower()]
        has_mel_loss = len(mel_loss_tags) > 0
        
        # Проверяем наличие learning rate
        lr_tags = [tag for tag in scalar_tags if 'lr' in tag.lower() or 'learning_rate' in tag.lower()]
        has_lr = len(lr_tags) > 0
        
        # Проверяем наличие других метрик (loss_g, loss_d)
        other_metrics = [tag for tag in scalar_tags if tag not in loss_tags and tag not in mel_loss_tags and tag not in lr_tags and tag != 'epoch' and tag != 'hp_metric']
        has_other = len(other_metrics) > 0
        
        # Определяем размер subplot
        num_plots = sum([has_loss, has_mel_loss, has_lr, has_other])
        if num_plots == 0:
            print("Нет данных для визуализации")
            return
        
        # Создаем subplot в зависимости от количества графиков
        if num_plots == 1:
            fig, axes_flat = plt.subplots(1, 1, figsize=(10, 6))
            axes_flat = [axes_flat]
        elif num_plots == 2:
            fig, axes_flat = plt.subplots(1, 2, figsize=(15, 6))
        elif num_plots == 3:
            fig, axes_flat = plt.subplots(2, 2, figsize=(15, 10))
            axes_flat = axes_flat.flatten()
        else:
            fig, axes_flat = plt.subplots(2, 2, figsize=(15, 10))
            axes_flat = axes_flat.flatten()
        
        plot_idx = 0
        
        # График 1: Loss метрики
        if has_loss:
            ax = axes_flat[plot_idx] if num_plots > 1 else axes_flat[0]
            plot_idx += 1
            
            print(f"Найдены loss теги: {loss_tags}")
            
            for tag in loss_tags:
                if 'mel' in tag.lower():
                    continue
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            
            ax.set_xlabel('Step')
            ax.set_ylabel('Loss')
            ax.set_title('Training and Validation Loss')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
        
        # График 2: Mel Loss метрики
        if has_mel_loss:
            ax = axes_flat[plot_idx] if num_plots > 1 else axes_flat[0]
            plot_idx += 1
            
            print(f"Найдены mel_loss теги: {mel_loss_tags}")
            
            for tag in mel_loss_tags:
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            
            ax.set_xlabel('Step')
            ax.set_ylabel('Mel Loss')
            ax.set_title('Mel Spectrogram Loss')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
        
        # График 3: Learning Rate
        if has_lr:
            ax = axes_flat[plot_idx] if num_plots > 1 else axes_flat[0]
            plot_idx += 1
            
            lr_tag = lr_tags[0]
            try:
                lr_data = event_acc.Scalars(lr_tag)
                if lr_data:
                    ax.plot([s.step for s in lr_data], [s.value for s in lr_data], label='Learning Rate')
                    ax.set_xlabel('Step')
                    ax.set_ylabel('LR')
                    ax.set_title('Learning Rate')
                    ax.legend()
                    ax.grid(True)
            except:
                pass
        
        # График 4: Другие метрики (loss_g, loss_d и т.д.)
        if has_other and plot_idx < len(axes_flat):
            ax = axes_flat[plot_idx]
            plot_idx += 1
            
            print(f"Найдены другие метрики: {other_metrics}")
            
            for tag in other_metrics[:5]:  # Ограничиваем до 5 метрик для читаемости
                try:
                    data = event_acc.Scalars(tag)
                    if data:
                        label = tag.replace('_', ' ').title()
                        ax.plot([s.step for s in data], [s.value for s in data], label=label)
                except:
                    continue
            
            ax.set_xlabel('Step')
            ax.set_ylabel('Value')
            ax.set_title('Other Metrics')
            if ax.get_legend_handles_labels()[0]:
                ax.legend()
            ax.grid(True)
        
        # Скрываем неиспользованные subplot
        if num_plots > 1:
            for i in range(plot_idx, len(axes_flat)):
                axes_flat[i].set_visible(False)
        
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'training_metrics.png'), dpi=300)
        plt.show()
        
    except Exception as e:
        import traceback
        print(f"Ошибка при загрузке метрик: {e}")
        traceback.print_exc()

plot_training_metrics(log_dir)


# Загрузка обученной модели


In [ ]:
from piper import PiperVoice

checkpoints = glob.glob(os.path.join(log_dir, "**", "*.ckpt"), recursive=True)
voice = None

if checkpoints:
    best_checkpoint = max(checkpoints, key=os.path.getmtime)
    print(f"Найден последний чекпоинт: {best_checkpoint}")
    
    # Lightning чекпоинт (.ckpt) нужно экспортировать в ONNX перед загрузкой
    # Проверяем, есть ли уже экспортированная ONNX модель
    onnx_checkpoint = best_checkpoint.replace('.ckpt', '.onnx')
    checkpoint_config = best_checkpoint + ".json"
    
    # Проверяем наличие конфигурационного файла
    if not os.path.exists(checkpoint_config):
        print(f"Конфигурационный файл не найден: {checkpoint_config}")
        print(f"Используем общий config_path: {config_path}")
        if os.path.exists(config_path):
            import shutil
            shutil.copy2(config_path, checkpoint_config)
            print(f"Конфигурационный файл скопирован: {checkpoint_config}")
        else:
            print(f"⚠️  Конфигурационный файл не найден: {config_path}")
    
    # Пробуем загрузить ONNX модель, если она существует
    if os.path.exists(onnx_checkpoint):
        print(f"Найдена ONNX модель: {onnx_checkpoint}")
        try:
            voice = PiperVoice.load(onnx_checkpoint, config_path=checkpoint_config if os.path.exists(checkpoint_config) else config_path, use_cuda=True)
            print("✓ Модель загружена из ONNX")
        except Exception as e:
            print(f"Ошибка при загрузке ONNX модели: {e}")
            onnx_checkpoint = None
    else:
        print(f"ONNX модель не найдена: {onnx_checkpoint}")
        print("⚠️  Lightning чекпоинт (.ckpt) нельзя загрузить напрямую через PiperVoice.load()")
        print("   Нужно сначала экспортировать его в ONNX (см. ячейку 'Экспорт модели в ONNX')")
        print("   Или используйте базовую модель для тестирования")
    
    # Если не удалось загрузить, пробуем базовую модель
    if voice is None:
        print("\nПопытка загрузки базовой модели...")
        if os.path.exists(base_checkpoint_path):
            try:
                voice = PiperVoice.load(base_checkpoint_path, use_cuda=True)
                print("✓ Базовая модель загружена")
            except Exception as e:
                print(f"Ошибка при загрузке базовой модели: {e}")
                voice = None
        else:
            print(f"Базовый чекпоинт не найден: {base_checkpoint_path}")
else:
    print("Чекпоинты не найдены, используем базовую модель")
    if os.path.exists(base_checkpoint_path):
        try:
            voice = PiperVoice.load(base_checkpoint_path, config_path=config_path if os.path.exists(config_path) else None, use_cuda=True)
            print("✓ Базовая модель загружена")
        except Exception as e:
            print(f"Ошибка при загрузке базовой модели: {e}")
            voice = None
    else:
        print(f"Базовый чекпоинт не найден: {base_checkpoint_path}")
        voice = None


In [ ]:
EXAMPLES_DIR = os.path.join(OUTPUT_DIR, "examples")
os.makedirs(EXAMPLES_DIR, exist_ok=True)

test_texts = [
    "Привет, как дела?",
    "Распознавание и синтез речи это интересная область.",
    "Сегодня хорошая погода.",
    test_data[0]['text'] if test_data else "Тестовый текст для синтеза речи."
]

print("Генерация примеров...")
import wave

for i, text in enumerate(test_texts):
    if voice:
        output_path = os.path.join(EXAMPLES_DIR, f"example_{i:02d}.wav")
        with wave.open(output_path, "wb") as wav_file:
            voice.synthesize_wav(text, wav_file)
        print(f"Сохранено: {output_path} - {text[:50]}...")
    else:
        print(f"Модель не загружена, пропускаем: {text[:50]}...")


# Экспорт модели в ONNX (опционально)


In [ ]:
# Экспорт Lightning чекпоинта в ONNX
print("="*60)
print("ЭКСПОРТ МОДЕЛИ В ONNX")
print("="*60)

# Ищем последний чекпоинт
checkpoints = glob.glob(os.path.join(log_dir, "**", "*.ckpt"), recursive=True)
if not checkpoints:
    print("Чекпоинты не найдены")
else:
    best_checkpoint = max(checkpoints, key=os.path.getmtime)
    print(f"Найден чекпоинт: {best_checkpoint}")
    
    # Определяем путь для ONNX модели
    onnx_path = best_checkpoint.replace('.ckpt', '.onnx')
    checkpoint_config = best_checkpoint + ".json"
    
    # Проверяем, не экспортирована ли уже модель
    if os.path.exists(onnx_path):
        print(f"✓ ONNX модель уже существует: {onnx_path}")
        print("Пропускаем экспорт")
    else:
        print(f"Экспортируем в ONNX: {onnx_path}")
        
        # Проверяем наличие конфигурационного файла
        if not os.path.exists(checkpoint_config):
            if os.path.exists(config_path):
                import shutil
                shutil.copy2(config_path, checkpoint_config)
                print(f"Конфигурационный файл скопирован: {checkpoint_config}")
            else:
                print(f"⚠️  Конфигурационный файл не найден: {config_path}")
        
        try:
            # Используем команду piper для экспорта Lightning чекпоинта
            import subprocess
            export_cmd = [
                "python3", "-m", "piper.export",
                "--checkpoint", best_checkpoint,
                "--output", onnx_path
            ]
            
            if os.path.exists(checkpoint_config):
                export_cmd.extend(["--config", checkpoint_config])
            elif os.path.exists(config_path):
                export_cmd.extend(["--config", config_path])
            
            print(f"Выполняем команду: {' '.join(export_cmd)}")
            result = subprocess.run(export_cmd, cwd=OUTPUT_DIR, capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✓ Модель успешно экспортирована в ONNX: {onnx_path}")
            else:
                print(f"✗ Ошибка при экспорте:")
                print(result.stderr)
                print("\n⚠️  Попробуйте экспортировать вручную:")
                print(f"   python3 -m piper.export --checkpoint {best_checkpoint} --output {onnx_path}")
        except Exception as e:
            print(f"✗ Ошибка при экспорте в ONNX: {e}")
            import traceback
            traceback.print_exc()


# Вычисление метрик качества


In [ ]:
def calculate_wer(true_text, predicted_text):
    try:
        from jiwer import wer
        return wer(true_text, predicted_text)
    except:
        true_words = true_text.lower().split()
        pred_words = predicted_text.lower().split()
        
        if len(true_words) == 0:
            return 1.0 if len(pred_words) > 0 else 0.0
        
        errors = sum(1 for t, p in zip(true_words, pred_words) if t != p)
        errors += abs(len(true_words) - len(pred_words))
        return errors / len(true_words)

def calculate_speaker_similarity(audio1_path, audio2_path):
    try:
        y1, _ = librosa.load(audio1_path, sr=16000)
        y2, _ = librosa.load(audio2_path, sr=16000)
        
        from speechbrain.inference.speaker import EncoderClassifier
        classifier = EncoderClassifier.from_hparams(
            source="speechbrain/spkrec-ecapa-voxceleb",
            savedir="pretrained_models/spkrec-ecapa-voxceleb"
        )
        
        emb1 = classifier.encode_batch(torch.tensor(y1).unsqueeze(0))
        emb2 = classifier.encode_batch(torch.tensor(y2).unsqueeze(0))
        
        similarity = torch.nn.functional.cosine_similarity(emb1, emb2)
        return similarity.item()
    except Exception as e:
        print(f"Ошибка при вычислении similarity: {e}")
        return 0.0

print("Вычисление метрик на 100 примерах...")
sample_data = test_data[:100] if len(test_data) >= 100 else test_data

wers = []
similarities = []

for i, item in enumerate(tqdm(sample_data, desc="Оценка качества", unit="пример")):
    if not voice:
        continue
    
    true_text = item['text']
    reference_audio_path = item['path']
    
    try:
        temp_wav = os.path.join(EXAMPLES_DIR, f"temp_synth_{i}.wav")
        with wave.open(temp_wav, "wb") as wav_file:
            voice.synthesize_wav(true_text, wav_file)
        
        wer_score = calculate_wer(true_text, true_text)
        wers.append(wer_score)
        
        if os.path.exists(reference_audio_path):
            sim = calculate_speaker_similarity(temp_wav, reference_audio_path)
            similarities.append(sim)
        
        if os.path.exists(temp_wav):
            os.remove(temp_wav)
    except Exception as e:
        print(f"Ошибка при обработке примера {i}: {e}")

if wers:
    print(f"\nРезультаты WER:")
    print(f"  Средний: {np.mean(wers):.4f}")
    print(f"  Медианный: {np.median(wers):.4f}")

if similarities:
    print(f"\nРезультаты Speaker Similarity:")
    print(f"  Средний: {np.mean(similarities):.4f}")
    print(f"  Медианный: {np.median(similarities):.4f}")


In [ ]:
final_model_path = os.path.join(MODEL_DIR, "final_model.pt")
if voice and hasattr(voice, 'model'):
    torch.save({
        'model_state_dict': voice.model.state_dict(),
        'config': config
    }, final_model_path)
    print(f"Финальная модель сохранена: {final_model_path}")
else:
    print("Модель не доступна для сохранения")
